# Mini-Project Solution

This notebook contains the solutions to the mini-project. **Please do not run the cells containing the code snippets, they are only for demonstration purposes.**

## Question 1

As we can see from the results, the performance in the test set is the same in both setups. However, in the train set, when applying dimensionality reduction the performance is slightly worse. This might be an indication that some information is lost after applying dimensionality reduction.

## Question 2

Changes should be done in **B. CODE TO BE CHANGED**. After the changes, the  code cell would look like this:

In [ ]:
# classifier hyperparameters
n_estimators = 30
max_depth = 4

###### B. CODE TO BE CHANGED ######
clf_type = 'rf' # change this to `lr` when using Logistic Regression 

# dimensionality reduction hyperparameters
n_neighbors = 30
min_dist = 0.5
n_components = 2


dim_red="tsne" # Change this to `tsne` when running

# initialize dimensionality reduction technique

umap = UMAP(n_components=2, n_neighbors=n_neighbors, \
                min_dist=min_dist, random_state=0)

data_after_dimensionality_reduction = \
    run_dimensionality_reduction(umap, dataset=dataset, numerical_cols=numerical_cols)

# initialize classifier
classifier = RandomForestClassifier(max_depth=max_depth,\
                                        n_estimators=n_estimators,\
                                            random_state=0)

# train classifier and visualize the results
run_classifier(classifier=classifier, clf_type=clf_type, \
    dataset=data_after_dimensionality_reduction, dim_red=dim_red)

###### END CODE TO BE CHANGED ######

## Question 3

Changes should be done in **A. CODE TO BE CHANGED**. After the changes, the  code cell would look like this:

In [ ]:
# classifier hyperparameters
n_estimators = 30
max_depth = 4

###### A. CODE TO BE CHANGED ######
clf_type = 'lr' # change this to `lr` when using Logistic Regression 

# initialize the classifier here
classifier = LogisticRegression()

# train classifier and visualize results
run_classifier(classifier=classifier, clf_type=clf_type, \
               dataset=dataset, dim_red="no_dim_red")

###### END CODE TO BE CHANGED ######

## Quesiton 4:

Changes should be done in **B. CODE TO BE CHANGED**. After the changes, the  code cell would look like this:

In [ ]:
# classifier hyperparameters
n_estimators = 30
max_depth = 4

###### B. CODE TO BE CHANGED ######
clf_type = 'lr' # change this to `lr` when using Logistic Regression 

# dimensionality reduction hyperparameters
n_neighbors = 30
min_dist = 0.5
n_components = 2


dim_red="tsne" # Change this to `tsne` when running

# initialize dimensionality reduction technique

tsne = TSNE(perplexity=20, random_state=0)

data_after_dimensionality_reduction = \
    run_dimensionality_reduction(tsne, dataset=dataset, numerical_cols=numerical_cols)

# initialize classifier
classifier = LogisticRegression()

# train classifier and visualize the results
run_classifier(classifier=classifier, clf_type=clf_type, \
    dataset=data_after_dimensionality_reduction, dim_red=dim_red)

###### END CODE TO BE CHANGED ######

## Question 5

Changes should be done in **B. CODE TO BE CHANGED**. After the changes, the  code cell would look like this:

In [ ]:
# classifier hyperparameters
n_estimators = 30
max_depth = 4

###### B. CODE TO BE CHANGED ######
clf_type = 'lr' # change this to `lr` when using Logistic Regression 

# dimensionality reduction hyperparameters
n_neighbors = 30
min_dist = 0.5
n_components = 2


dim_red="umap" # Change this to `tsne` when running

# initialize dimensionality reduction technique

umap = UMAP(perplexity=20, random_state=0)

data_after_dimensionality_reduction = \
    run_dimensionality_reduction(umap, dataset=dataset, numerical_cols=numerical_cols)

# initialize classifier
classifier = LogisticRegression()

# train classifier and visualize the results
run_classifier(classifier=classifier, clf_type=clf_type, \
    dataset=data_after_dimensionality_reduction, dim_red=dim_red)

###### END CODE TO BE CHANGED ######

## Tables of Area Under the Curve ROC


| Train set results | No dim. red. | UMAP | tSNE |
| -------- | -------- | -------- | -------- |
| Logistic Regression   | 0.99   | 0.89   | 0.91  |
| Random Forest   | 1.0   | 0.99   | 0.99   |

<br>
<br>

| Test set results | No dim. red. | UMAP | tSNE |
| -------- | -------- | -------- | -------- |
| Logistic Regression   | 0.99   | 0.89   | 0.89   |
| Random Forest   | 0.98   | 0.98  | 0.99   |




## Question 6:

When considering the test set, Random Forest performs better than Logistic regression in all cases, except when no dimensionality reduction is applied. In addition, we see that dimensionality reduction is not helpful in the case of Logistic Regression, while `tSNE` with Random Forest has a near perfect performance. The best set ups are: Logistic Regression without dimensionality reduction and Random Forest with `tSNE`. There are not  indications of overfitting since the performance metrics in the train and test sets are similar.